# Image deblurring

**Authors: M. Ravasi, D. Vargas, I. Vasconcelos**

Welcome to the second session of our **Solving large-scale inverse problems in Python with PyLops** tutorial!

Throughout this tutorial, we aim at developing the following aspects in image processing:

- We connect with the concepts learned in the previous session by defining a simple PyLops operator implementing a convolutional kernel for image processing  
- Learn how to use build-in PyLops operators for image manipulation including, blurring, sharpening, and edge detection
- Demonstrate the versatility of the linear operators when implemented in the solution of inverse problems, in particular, the image deblurring problem
- Analyze the influence of noise in contrast with the performance of different solvers, least-squares, TV-Regularization, and FISTA

## Useful links

- Tutorial Github repository: https://github.com/mrava87/pylops_pydata2020
        
- PyLops Github repository: https://github.com/equinor/pylops

- PyLops reference documentation: https://pylops.readthedocs.io/en/latest/

# Deblurring Images

Compact digital cameras were introduced in the early seventies thanks to the development of CCD sensors allowing the recording and storage of images in digital format. The goal of an image process is to capture an accurate representation of a particular scene; however, any image is influenced by the optical system itself and the overall effect is a blurred image reconstruction, and specialized algorithms performing deblurring need to be implemented in order to achieve a sharper image. Image deblurring is a method that aims at recovering the original sharp-image by removing effect caused by limited aperture, lens aberrations, defocus, and unintended motions. This can be done by defining a mathematical model of the blurring process with the idea of removing from the image the blurring effects.

Traditionally, the main assumption is that the blurring process is linear. When this is the case, a 2-dimensional convolutional model describes the imaging problem. The blurred Image (output) is the result of a point-spread function acting on a target sharp image (input). Such process is described by following transformation, 

\begin{equation}
g(x, y) = \int_{-\infty}^{\infty}\int_{-\infty}^{\infty} h(x-x_0,y-y_0) f(x_0, y_0) dx_0 dy_0
\end{equation}

This operation can be discretized as follows

\begin{equation}
g[i,n] = \sum_{j=-\infty}^{\infty} \sum_{m=-\infty}^{\infty} h[i-j,n-m] f[j,m] 
\end{equation}

and also implemented in the frequency domain using the convolution theorem

\begin{equation}
G(k_x, k_y) = \mathscr{F} (h(x,y)) * \mathscr{F} (f(x,y))
\end{equation}

The previous definition is an specific case of the more general problem $\mathbf{y} =  \mathbf{A} \mathbf{x}$ (*forward operation*), where $\mathbf{A}:\mathbb{F}^m \to \mathbb{F}^n$ is a linear operator that maps a vector of size $m$ in the *model space* to a vector of size $n$ in the *data space*, as explained in the previous session. The (*adjoint operation*) is $\mathbf{x} = \mathbf{A}^H \mathbf{y}$. In this context, our deblurring operation is an inverse problem removing the effect of the operator/matrix $\mathbf{A}$ from the data $\mathbf{y}$ to retrieve the model $\mathbf{x}$, i.e., $\hat{\mathbf{x}} = \mathbf{A}^{-1} \mathbf{y}$. 

This Jupyter Notebook is intended to guide you through the main steps in the implementation of 2d blurring and deblurring using the <code>pylops.signalprocessing.Convolve2D</code> operator, in this case we assume to know the structure of the point-spread function for an specific problem.



In [ ]:
%matplotlib inline

import numpy as np
from skimage import data
import matplotlib.pyplot as plt
from scipy.signal import convolve, correlate
import scooby

from pylops.optimization import leastsquares, sparsity
from pylops.signalprocessing import Convolve2D
from pylops import FirstDerivative, Laplacian
from pylops import LinearOperator
from pylops.utils import dottest

# Loading an image

We start by uploading an image. Here we use the **scikit-image** module <code>skimage.data</code> containing standard test images. This is were you set images with different features to analyze the performance of image deblurring and denoising in the next cells. Feel free to play around and even upload your own pictures.  

In [ ]:
image = data.camera()
# image = data.binary_blobs()
# image = data.checkerboard()
ny, nx = image.shape

plt.imshow(image)
plt.title("True image")

# Defining a 2D convolution Pylops operator



In [ ]:
class my_convolve2D(LinearOperator):
    def __init__(self, A, dims, dtype="float64"):
        self.A = A
        self.dims = dims
        self.shape = (np.prod(self.dims), np.prod(self.dims))
        self.dtype = np.dtype(dtype)

    def _matvec(self, x):
        x = np.reshape(x, self.dims)
        y = convolve(x, self.A, mode='same')
        return np.ndarray.flatten(y)

    def _rmatvec(self, x):
        x = np.reshape(x, self.dims)
        y = correlate(x, self.A, mode='same')
        return np.ndarray.flatten(y)

# Blurring an image

Now that we have defined our simple PyLops operator performing convolutions, it is time to actually apply the filter to the loaded image. Before we do that, let's first see if under the given parameters this linear operator passes the dot test. Once we make sure that this is the case, we can compute the convolution using NumPy matrix-like syntax. In the cell below, we select the simplest point spread function, the identity kernel. It should not blur the image and the error of the output compared to the input should be very low (machine precession)

In [ ]:
# Identity kernel
kernel = np.zeros([3, 3])
kernel[1, 1] = 1
C_op = my_convolve2D(A=kernel, dims=(ny, nx))

# Dot test
dottest(C_op, ny*nx, ny*nx, verb=True)


img_blur = C_op * image.flatten()
img_blur = img_blur.reshape(image.shape)


# PLOTTING
fig, ax = plt.subplots(1, 3, figsize=(15, 5))

ax[0].imshow(image)
ax[0].set_title('Input image')

ax[1].imshow(img_blur)
ax[1].set_title('Respose to identity')

him = ax[2].imshow(img_blur-image)
ax[2].set_title('Error output-input')
fig.colorbar(him, ax=ax[2])

# Box blur

$$ 
\mathbf{A} = \frac{1}{9}
\begin{bmatrix}
    1 & 1 & 1\\
    1 & 1 & 1\\
    1 & 1 & 1
\end{bmatrix} 
$$

In [ ]:
lengh = 3    # Play with the operators extention
kernel_box = 1/(lengh**2) * np.ones([lengh, lengh])

# Define the PyLops blurring Operator
Box_op = Convolve2D(N=ny*nx,
                    h=kernel_box,
                    offset=(kernel_box.shape[0]//2, kernel_box.shape[1]//2),
                    dims=(ny, nx))

img_box = Box_op * image.flatten()
img_box = img_box.reshape(image.shape)

# PLOTTING
fig, ax = plt.subplots(1, 3, figsize=(15, 5))

ax[0].imshow(image)
ax[0].set_title('Image')

ax[1].imshow(kernel_box)
ax[1].set_title('Kernel')

ax[2].imshow(img_box)
ax[2].set_title('Blurred Image')

# Edge detection - Sobel operator

Vertical derivative

$$ 
\mathbf{G_x} = 
\begin{bmatrix}
    1 & 0 & -1\\
    2 & 0 & -2\\
    1 & 0 & -1
\end{bmatrix} 
$$

Horizontal derivative

$$ 
\mathbf{G_y} = 
\begin{bmatrix}
    1 & 2 & 1\\
    0 & 0 & 0\\
    -1 & -2 & -1
\end{bmatrix} 
$$

gradient magnitude

$$
\mathbf{G} = \sqrt{\mathbf{G_x}^2 + \mathbf{G_x}^2} 
$$

In [ ]:
# horizontal edge detector
kernel_h = np.array([[1, 0, -1],
                     [2, 0, -2],
                     [1, 0, -1]])

# vertical edge detector
kernel_v = np.array([[1, 2, 1],
                     [0, 0, 0],
                     [-1, -2, -1]])

# Define the PyLops blurring Operator
Sobel_x_op = Convolve2D(N=ny*nx,
                        h=kernel_h,
                        offset=(kernel_h.shape[0]//2, kernel_h.shape[1]//2),
                        dims=(ny, nx))
Sobel_y_op = Convolve2D(N=ny*nx,
                        h=kernel_v,
                        offset=(kernel_v.shape[0]//2, kernel_v.shape[1]//2),
                        dims=(ny, nx))

Gx = Sobel_x_op * image.flatten()
Gy = Sobel_y_op * image.flatten()
Gx = Gx.reshape(image.shape)
Gy = Gy.reshape(image.shape)

# Gradient magnitude
G = np.sqrt(Gy**2 + Gy**2)

# PLOTTING
fig, ax = plt.subplots(1, 3, figsize=(15, 5))

ax[0].imshow(image)
ax[0].set_title('Image')

ax[1].imshow(kernel_v)
ax[1].set_title('Kernel')

ax[2].imshow(G)
ax[2].set_title('Image edges')


# Gaussian filter*

While discussing filtering in the spatial domain we mentioned a filtering mask, which is used to convolute with the image. These masks are generated depending on their purpose, such as smoothing, edge detection, edge enhancement, etc. The filters are decided by a given size $m \times n$ and with the use of a mathematical function one can calculate the values of the filter discretely. We will later see that the nature of the function used will result in how well the filter will perform. 

Using a Gaussian filter, also known as Gaussian smoothing, is an operator that is used to blur images and remove detail and noise. This is similar to the way a mean filter works, but the Gaussian filter uses a different kernel. This kernel is represented with a Gaussian bell shaped bump. This kernel has some special properties regarding separability that we will look at in detail.

\begin{equation}
G(x) = \frac{1}{\sqrt{2 \pi } \sigma} e^{- \frac{x^{2}}{2 \sigma ^{2}}}
\label{1dgaussian}
\end{equation}

\begin{equation}
G(x,y) = \frac{1}{2 \pi \sigma ^{2}} e^{- \frac{x^{2} + y^{2}}{2 \sigma ^{2}}}
\label{2dgaussian}
\end{equation}


In [ ]:
# Play around with this parameters
mx, my = 25, 15                 # 25, 15
sigma_x, sigma_y = 4.5, 2.5     # 4.5, 2.5

x = np.linspace(-mx//2, mx//2, mx)
y = np.linspace(-my//2, my//2, my)[..., None]
kernel_gauss = 1/(2*np.pi*sigma_x*sigma_y) * \
    np.exp(-(x**2/(2*sigma_x**2) + y**2/(2*sigma_y**2)))

# Define the PyLops blurring Operator
Gauss_op = Convolve2D(N=ny*nx,
                      h=kernel_gauss,
                      offset=(kernel_gauss.shape[0]//2,
                              kernel_gauss.shape[1]//2),
                      dims=(ny, nx))

img_gauss = Gauss_op * image.flatten()
img_gauss_ = img_gauss.reshape(image.shape)

# PLOTTING
fig, ax = plt.subplots(1, 3, figsize=(15, 5))

ax[0].imshow(image)
ax[0].set_title('Image')

ax[1].imshow(kernel_gauss)
ax[1].set_title('Gaussian Kernel')

him2 = ax[2].imshow(img_gauss_)
ax[2].set_title('Blurred Image')

# Image Deblurring 

Finally, we consider a standard deblurring case. Here we assume a picture was taken in real life; therefore, a whole set of potential artifacts are present in it. Our mission is to try and reconstruct a shaper image representation of the scene by removing the blurring effects. Luckily, we assume to have some information on the optical system and decided to implement an inverse problem. Depending on the application, different solvers are available, below we will compare the performance of some well-known algorithms to deblur the image and compare it with its original version.

A set of ready to use solvers are implemented in the PyLops module <code>pylops.optimization.leastsquares<code> and <code>pylops.optimization.sparsity<code>. Here we attempt debluring by,

    - least squares inversion
    - least squares - regularized inversion
    - TV inversion
    - FISTA inversion

In [ ]:
# L2 regularization operator - Second derivative
D2op = Laplacian(dims=(ny, nx), dtype=np.float)
# L1 regularization operator  - First derivative
Dop = [FirstDerivative(ny * nx, dims=(ny, nx), dir=0),
       FirstDerivative(ny * nx, dims=(ny, nx), dir=1)]

# least squares inversion
deblur_l2 = leastsquares.NormalEquationsInversion(Op=Gauss_op,
                                                  Regs=None,
                                                  data=img_gauss,
                                                  epsI=0,
                                                  maxiter=40)

# least squares - regularized inversion
deblur_l2_reg = leastsquares.RegularizedInversion(Op=Gauss_op,
                                                  Regs=[D2op],
                                                  data=img_gauss,
                                                  epsRs=[1e0],
                                                  show=True,
                                                  **dict(iter_lim=20))

# TV inversion
deblur_tv = sparsity.SplitBregman(Op=Gauss_op,
                                  RegsL1=Dop,
                                  data=img_gauss,
                                  niter_outer=10,
                                  niter_inner=5,
                                  mu=1.8,
                                  epsRL1s=[1e-1, 1e-1],
                                  tol=1e-4,
                                  tau=1.,
                                  ** dict(iter_lim=5, damp=1e-4))[0]

# FISTA inversion
deblur_fista = sparsity.FISTA(Op=Gauss_op,
                              data=img_gauss,
                              eps=1e-1,
                              niter=100)[0]

# Reshape images
deblur_l2 = deblur_l2.reshape(image.shape)
deblur_l2_reg = np.real(deblur_l2_reg.reshape(image.shape))
deblur_tv = deblur_tv.reshape(image.shape)
deblur_fista = deblur_fista.reshape(image.shape)


# PLOTTING
fig, ax = plt.subplots(2, 3, sharex=True, sharey=True, figsize=(12, 8))

ax[0, 0].imshow(image, aspect='auto')
ax[0, 0].set_title('Original Image')

ax[0, 1].imshow(deblur_l2, aspect='auto')
ax[0, 1].set_title('LS-Inversion')

ax[0, 2].imshow(deblur_l2_reg, aspect='auto')
ax[0, 2].set_title('Regularized LS-Inversion')

ax[1, 0].imshow(img_gauss_, aspect='auto')
ax[1, 0].set_title('Blurred Image')

ax[1, 1].imshow(deblur_tv, aspect='auto')
ax[1, 1].set_title('TV-Inversion')

ax[1, 2].imshow(deblur_fista, aspect='auto')
ax[1, 2].set_title('FISTA Inversion')
fig.tight_layout()

# Image Deblurring - The effect of noise Noise

We have so far discussed the blurring of images. In practical image processing, observed images are commonly polluted with noise. The nature of the noise is influenced by multiple sources, and mathematically can be linear, nonlinear, multiplicative, and additive. In this exercise, we consider common additive noise, which can be included in our linear model by a term perturbing the blurred image 

$$
\mathbf{B} = \mathbf{B_{blur}} + \mathbf{N}
$$

Below we add Gaussian white noise with a given standard deviation to our blurred image and perform image deblurring as in the cell above

In [ ]:
scale = 5.8
noise = np.random.normal(0, scale, img_gauss_.shape)
img_gauss_noisy_ = img_gauss_ + noise
img_gauss_noisy = img_gauss_noisy_.flatten()


# PLOTTING
fig, ax = plt.subplots(1, 3, figsize=(15, 5))

ax[0].imshow(img_gauss_)
ax[0].set_title('Blurred Image')

ax[1].imshow(img_gauss_noisy_)
ax[1].set_title('Blurred Image + Noise')

him2 = ax[2].imshow(img_gauss_noisy_ - img_gauss_)
ax[2].set_title('Noise level')


In [ ]:

# least squares inversion
deblur_l2 = leastsquares.NormalEquationsInversion(Op=Gauss_op,
                                                  Regs=None,
                                                  data=img_gauss_noisy,
                                                  epsI=0,
                                                  maxiter=40)

# least squares - regularized inversion
deblur_l2_reg = leastsquares.RegularizedInversion(Op=Gauss_op,
                                                  Regs=[D2op],
                                                  data=img_gauss_noisy,
                                                  epsRs=[1e0],
                                                  show=True,
                                                  **dict(iter_lim=20))

# TV inversion
deblur_tv = sparsity.SplitBregman(Op=Gauss_op,
                                  RegsL1=Dop,
                                  data=img_gauss_noisy,
                                  niter_outer=10,
                                  niter_inner=5,
                                  mu=1.8,
                                  epsRL1s=[1e-1, 1e-1],
                                  tol=1e-4,
                                  tau=1.,
                                  ** dict(iter_lim=5, damp=1e-4))[0]

# FISTA inversion
deblur_fista = sparsity.FISTA(Op=Gauss_op,
                              data=img_gauss_noisy,
                              eps=1e-1,
                              niter=100)[0]

# Reshape images
deblur_l2 = deblur_l2.reshape(image.shape)
deblur_l2_reg = np.real(deblur_l2_reg.reshape(image.shape))
deblur_tv = deblur_tv.reshape(image.shape)
deblur_fista = deblur_fista.reshape(image.shape)


# PLOTTING
fig, ax = plt.subplots(2, 3, sharex=True, sharey=True, figsize=(12, 8))

ax[0, 0].imshow(image, aspect='auto')
ax[0, 0].set_title('Original Image')

ax[0, 1].imshow(deblur_l2, aspect='auto')
ax[0, 1].set_title('LS-Inversion')

ax[0, 2].imshow(deblur_l2_reg, aspect='auto')
ax[0, 2].set_title('Regularized LS-Inversion')

ax[1, 0].imshow(img_gauss_, aspect='auto')
ax[1, 0].set_title('Blurred Image')

ax[1, 1].imshow(deblur_tv, aspect='auto')
ax[1, 1].set_title('TV-Inversion')

ax[1, 2].imshow(deblur_fista, aspect='auto')
ax[1, 2].set_title('FISTA Inversion')
fig.tight_layout()

# Recap

In this tutorial we have learned to:

- Define a simple PyLops operator implementing a convolutional kernel for image processing  
- Use build-in PyLops operators for image manipulation including
- Solve the image deblurring problem using different inverse problems schemes
- Denoise a blurred image with the help of least-squares, TV-Regularization, and FISTA


In [ ]:
scooby.Report(core='pylops')